In [1]:
!pip freeze | grep sci

scikit-learn==1.0.2
scipy==1.8.1


In [2]:
import pickle

import pandas as pd

from sklearn.feature_extraction import DictVectorizer
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.pipeline import make_pipeline

In [3]:
import mlflow

mlflow.set_tracking_uri("http://127.0.0.1:5000")
mlflow.set_experiment("green-taxi-duration")

<Experiment: artifact_location='s3://nrf-mlflow/1', experiment_id='1', lifecycle_stage='active', name='green-taxi-duration', tags={}>

In [4]:
def read_dataframe(filename: str):
    df = pd.read_parquet(filename)

    df['duration'] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
    df.duration = df.duration.dt.total_seconds() / 60
    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)
    return df


def prepare_dictionaries(df: pd.DataFrame):
    df['PU_DO'] = df['PULocationID'] + '_' + df['DOLocationID']
    categorical = ['PU_DO']
    numerical = ['trip_distance']
    dicts = df[categorical + numerical].to_dict(orient='records')
    return dicts

In [5]:
df_train = read_dataframe('../../data/green_tripdata_2021-01.parquet')
df_val = read_dataframe('../../data/green_tripdata_2021-02.parquet')

target = 'duration'
y_train = df_train[target].values
y_val = df_val[target].values

dict_train = prepare_dictionaries(df_train)
dict_val = prepare_dictionaries(df_val)

In [6]:
with mlflow.start_run():
    params = dict(max_depth=20, n_estimators=100, min_samples_leaf=10, random_state=0)
    mlflow.log_params(params)

    pipeline = make_pipeline(
        DictVectorizer(),
        RandomForestRegressor(**params, n_jobs=-1)
    )

    pipeline.fit(dict_train, y_train)
    y_pred = pipeline.predict(dict_val)

    rmse = mean_squared_error(y_pred, y_val, squared=False)
    print(params, rmse)
    mlflow.log_metric('rmse', rmse)

    mlflow.sklearn.log_model(pipeline, artifact_path="model")

{'max_depth': 20, 'n_estimators': 100, 'min_samples_leaf': 10, 'random_state': 0} 6.7558229919200725


/Users/nicolasroldan/.local/share/virtualenvs/streaming-AxiVUBvF/lib/python3.8/site-packages/_distutils_hack/__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")


In [7]:
from mlflow.tracking import MlflowClient

In [9]:
MLFLOW_TRACKING_URI = 'http://127.0.0.1:5000'
RUN_ID = '66679dc47de544cdae3289642deb8eb5'

client = MlflowClient(tracking_uri=MLFLOW_TRACKING_URI)

In [10]:
path = client.download_artifacts(run_id=RUN_ID, path='model', dst_path=".")

In [11]:
with open("model/model.pkl", 'rb') as f_out:
    model = pickle.load(f_out)

In [12]:
logged_model = f's3://nrf-mlflow/1/{RUN_ID}/artifacts/model'
model = mlflow.pyfunc.load_model(logged_model)

In [13]:
model

mlflow.pyfunc.loaded_model:
  artifact_path: model
  flavor: mlflow.sklearn
  run_id: 66679dc47de544cdae3289642deb8eb5

In [14]:
mlflow.pyfunc.get_model_dependencies(logged_model)

2022/07/03 16:46:31 INFO mlflow.pyfunc: To install the dependencies that were used to train the model, run the following command: '%pip install -r /var/folders/0s/qfqb94xj5pg6xz3d_l7xfsqm0000gn/T/tmpdmqouvp6/requirements.txt'.


'/var/folders/0s/qfqb94xj5pg6xz3d_l7xfsqm0000gn/T/tmpdmqouvp6/requirements.txt'